In [1]:
import time
import warnings
warnings.filterwarnings('ignore') # Hide warnings that can generate and clutter notebook


def timing_experiment(x, y, distance_callable, distance_params = None, average=30):
    """Time the average time it takes to take the distance from the first time series
    to all of the other time series in X.

    Parameters
    ----------
    X: np.ndarray
        A dataset of time series.
    distance_callable: Callable
        A callable that is the distance function to time.

    Returns
    -------
    float
        Average time it took to run a distance
    """
    if distance_params is None:
        distance_params = {}
    total_time = 0
    for i in range(0, average):
        start = time.time()
        curr_dist = distance_callable(x, y, **distance_params)
        total_time += time.time() - start

    return total_time / average


In [4]:
from sktime.distances.tests._utils import create_test_distance_numpy

from sktime.distances import distance_factory
from tslearn.metrics import dtw as tslearn_dtw
from dtw import dtw as dtw_python_dtw
from rust_dtw import dtw as rust_dtw_dtw
import pandas as pd

sktime_timing = []
tslearn_timing = []
dtw_python_timing = []
rust_dtw_timing = []
numba_sktime_timing = []

col_headers = []

univariate_timings_df = None

def univariate_experiment():
    increment_timepoints = 1000
    starting_num_timepoints = 1000
    ending_num_timepoints = 10000

    for i in range(starting_num_timepoints, ending_num_timepoints + increment_timepoints, increment_timepoints):
        col_headers.append(i)
        distance_m_d = create_test_distance_numpy(2, 1, i)

        x = distance_m_d[0][0]
        y = distance_m_d[1][0]
        numba_sktime = distance_factory(x, y, metric='dtw')

        tslearn_time = timing_experiment(x, y, tslearn_dtw)
        dtw_python_time = timing_experiment(x, y, dtw_python_dtw)
        rust_dtw_time = timing_experiment(x, y, rust_dtw_dtw, {'window':i, 'distance_mode': 'euclidean'})
        sktime_time = timing_experiment(distance_m_d[0], distance_m_d[1], numba_sktime)

        sktime_timing.append(sktime_time)
        tslearn_timing.append(tslearn_time)
        dtw_python_timing.append(dtw_python_time)
        rust_dtw_timing.append(rust_dtw_time)


    uni_df = pd.DataFrame({
        'time points': col_headers,
        'sktime': sktime_timing,
        'tslearn': tslearn_timing,
        'rust-dtw': rust_dtw_timing,
        'dtw-python': dtw_python_timing
    })
    return uni_df

univariate_timings_df = univariate_experiment()

In [11]:
from sktime.distances.tests._utils import create_test_distance_numpy

from sktime.distances import distance_factory
from tslearn.metrics import dtw as tslearn_dtw
import pandas as pd

multivariate_timings_df = None

sktime_timing = []
tslearn_timing = []

col_headers = []

def multivariate_experiment():
    increment_timepoints = 100
    starting_num_timepoints = 100
    ending_num_timepoints = 500

    for i in range(starting_num_timepoints, ending_num_timepoints + increment_timepoints, increment_timepoints):
        col_headers.append(i)
        distance_m_d = create_test_distance_numpy(2, i, i)

        x = distance_m_d[0]
        y = distance_m_d[1]
        tslearn_x = x.reshape((x.shape[1], x.shape[0])) # tslearn wants m, d format
        tslearn_y = y.reshape((y.shape[1], y.shape[0])) # tslearn wants m, d format
        numba_sktime = distance_factory(x, y, metric='dtw')

        tslearn_time = timing_experiment(tslearn_x, tslearn_y, tslearn_dtw)
        sktime_time = timing_experiment(x, y, numba_sktime)

        sktime_timing.append(sktime_time)
        tslearn_timing.append(tslearn_time)


    multi_df = pd.DataFrame({
        'time points': col_headers,
        'sktime': sktime_timing,
        'tslearn': tslearn_timing,
    })
    return multi_df

multivariate_timings_df = multivariate_experiment()

In [18]:
print(univariate_timings_df)

   time points    sktime   tslearn  rust-dtw  dtw-python
0         1000  0.014600  0.007733  0.006067    0.037633
1         2000  0.040167  0.033200  0.025167    0.146033
2         3000  0.083433  0.073533  0.056967    0.325900
3         4000  0.148467  0.133500  0.103133    0.583167
4         5000  0.223367  0.217200  0.158433    0.919367
5         6000  0.321967  0.303000  0.230767    1.296900
6         7000  0.433367  0.396533  0.315900    1.743367
7         8000  0.599733  0.505367  0.404708    2.274967
8         9000  0.729467  0.664900  0.521933    2.959800
9        10000  0.897800  0.822767  0.643067    3.653133


In [19]:
print(multivariate_timings_df)

   time points    sktime   tslearn
0          100  0.004700  0.000667
1          200  0.012967  0.005467
2          300  0.038467  0.017950
3          400  0.087000  0.042933
4          500  0.204400  0.084633
